In [1]:
import sys
sys.path.append('../')

from envs.GridMaps import *
from utilities.transitionDatasetGeneration import *

In [2]:
maze = maze14x14["maze"]
n_epochs = 3000
n_models = 3
hidden_size = 32

folder_name = "proeba"  #to store the models weights and losses

train_transitions, val_transitions = transition_generator(maze)

In [3]:
models_arr = []

for i in range(n_models):
    print(f"Training model {i+1}...")

    model = ModelTrainer(train_transitions, val_transitions, n_epochs, hidden_size, rand_seed=True)

    model.fit()
    models_arr.append(model)

Training model 1...


100%|██████████| 3000/3000 [00:39<00:00, 75.88it/s]


Trained successfully!
Final loss: 0.04545470327138901

Training model 2...


100%|██████████| 3000/3000 [00:38<00:00, 78.17it/s]


Trained successfully!
Final loss: 0.04557742923498154

Training model 3...


100%|██████████| 3000/3000 [00:36<00:00, 82.52it/s]

Trained successfully!
Final loss: 0.09438440203666687



In [4]:
store_models(models_arr, folder_name)

In [5]:
#Choose the inputs [y,x,action]
inp1 = [5, 3, 3]
inp2 = [5, 4, 3]

In [6]:
print('\nEVALUATION:\n')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

arr1 = []
arr1_raw = []
arr2 = []
arr2_raw = []

for model in models_arr:
    # Single predictions
    with torch.no_grad():
        # Move the input data to the GPU
        inp1 = torch.FloatTensor([inp1]).to(device)
        inp2 = torch.FloatTensor([inp2]).to(device)
        
        # Perform inference on the GPU
        resultado1 = model.model(inp1)
        resultado2 = model.model(inp2)
        
        # Move the result back to CPU and convert to numpy
        resultado1 = resultado1.cpu().detach().numpy()
        resultado2 = resultado2.cpu().detach().numpy()

        arr1.append(np.int32(np.round(resultado1[0])))
        arr1_raw.append(resultado1[0])
        arr2.append(np.int32(np.round(resultado2[0])))
        arr2_raw.append(resultado2[0])
        
for elem, elem2 in zip(arr1, arr2):
    print(elem, '         ', elem2)

#whats the mean and the std of the raw values?
print('\n')
print('Mean of raw values:')
print(np.mean(arr1_raw, axis=0))
print(np.mean(arr2_raw, axis=0))
print('\n')
print('Std of raw values:')
print(np.std(arr1_raw, axis=0), '     ', np.linalg.norm(np.std(arr1_raw, axis=0)))
print(np.std(arr2_raw, axis=0), '     ', np.linalg.norm(np.std(arr2_raw, axis=0)))



EVALUATION:

Using device: cuda


ValueError: only one element tensors can be converted to Python scalars